In [111]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [112]:
#generate categorical variables list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [113]:
#check # of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [114]:
charity_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [115]:
#Drop EIN, name, and Classification features
charity_df_mod_pre = charity_df.drop(['CLASSIFICATION','NAME','EIN','STATUS','APPLICATION_TYPE'],1)

#Remove rows with 0 income_amt
#charity_df_mod = charity_df_mod_pre[charity_df_mod_pre['INCOME_AMT'] != '0']
charity_df_mod = charity_df_mod_pre
charity_df_mod.head()

,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,0,N,5000,1
1,Independent,Preservation,Co-operative,1-9999,N,108590,1
2,CompanySponsored,ProductDev,Association,0,N,5000,0
3,CompanySponsored,Preservation,Trust,10000-24999,N,6692,1
4,Independent,Heathcare,Trust,100000-499999,N,142590,1


In [116]:
charity_df_mod_pre.shape

(34299, 7)

In [117]:
charity_df_mod.shape


(34299, 7)

In [118]:
#create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

#generate categorical variables list for mod
charity_cat_mod = charity_df_mod.dtypes[charity_df_mod.dtypes == "object"].index.tolist()

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df_mod[charity_cat_mod]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat_mod)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [119]:
encode_df.shape

(34299, 26)

In [120]:
# Merge one-hot encoded features and drop the originals
charity_df_mod = charity_df_mod.merge(encode_df,left_index=True, right_index=True)
charity_df_mod = charity_df_mod.drop(charity_cat_mod,1)
charity_df_mod.head()

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [121]:
# Split our preprocessed data into our features and target arrays
y = charity_df_mod["IS_SUCCESSFUL"].values
X = charity_df_mod.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [122]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Deep neural network

In [123]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 8)                 224       
_________________________________________________________________
dense_37 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 6         
Total params: 275
Trainable params: 275
Non-trainable params: 0
_________________________________________________________________


In [124]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [125]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.7321 - acc: 0.4880: 3s - loss: 0.91
Epoch 2/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.6912 - acc: 0.5321
Epoch 3/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6912 - acc: 0.5321
Epoch 4/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.6912 - acc: 0.5321
Epoch 5/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6911 - acc: 0.5321
Epoch 6/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6912 - acc: 0.5321
Epoch 7/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6912 - acc: 0.5321
Epoch 8/100
25724/25724 [==============================] - 2s 72us/sample - loss: 0.6912 - acc: 0.53210s - loss
Epoch 9/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.6912 - acc: 0.5321
Epoch 10/100
25724/25724 [=====================

25724/25724 [==============================] - 1s 52us/sample - loss: 0.6911 - acc: 0.5321
Epoch 79/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.6912 - acc: 0.5321
Epoch 80/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6912 - acc: 0.53210s - loss: 0.6914 - acc: 0
Epoch 81/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.6911 - acc: 0.5321
Epoch 82/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6912 - acc: 0.5321
Epoch 83/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6912 - acc: 0.5321
Epoch 84/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.6911 - acc: 0.5321
Epoch 85/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.6912 - acc: 0.5321
Epoch 86/100
25724/25724 [==============================] - 2s 72us/sample - loss: 0.6912 - acc: 0.5321
Epoch 87/100
25724/25724 [=========================

In [126]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6910 - acc: 0.5332
Loss: 0.6909521685258293, Accuracy: 0.5331778526306152


## Single layer network

In [127]:
# Single layer model create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim=number_input_features))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 1)                 28        
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 2         
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [128]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [129]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.6532 - acc: 0.6230
Epoch 2/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6270 - acc: 0.6864
Epoch 3/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.6172 - acc: 0.6882
Epoch 4/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.6123 - acc: 0.6903
Epoch 5/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.6097 - acc: 0.6990
Epoch 6/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6085 - acc: 0.6994
Epoch 7/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6078 - acc: 0.6993
Epoch 8/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6075 - acc: 0.6994
Epoch 9/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.6072 - acc: 0.6995
Epoch 10/100
25724/25724 [==============================] - 1s 40us/sampl

25724/25724 [==============================] - 1s 46us/sample - loss: 0.6051 - acc: 0.7005
Epoch 80/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6051 - acc: 0.7004
Epoch 81/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6051 - acc: 0.7004
Epoch 82/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6051 - acc: 0.7004
Epoch 83/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6051 - acc: 0.7005
Epoch 84/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6051 - acc: 0.7005
Epoch 85/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6051 - acc: 0.7004
Epoch 86/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6051 - acc: 0.7005
Epoch 87/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6051 - acc: 0.7005
Epoch 88/100
25724/25724 [==============================] - 1s 57us/sample - 

In [130]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6083 - acc: 0.6946
Loss: 0.6082554299073734, Accuracy: 0.6945772767066956
